
# Processamento de Imagens - Cutting

Este notebook tem como objetivo processar as imagens anotadas no CVAT e aplicar as máscaras nas áreas específicas dos frames, redimensionando as imagens para o formato desejado. Este é o segundo passo no pipeline, onde, após as máscaras serem geradas manualmente, o código converte as anotações XML em imagens processadas.


## Garantia da Diversidade
Garantir a diversidade das imagens foi uma prioridade durante todo o processo. Para evitar redundância, não utilizamos frames sequenciais e selecionamos frames com intervalos que capturam variações significativas no cenário e nos contextos dos vídeos. Abaixo, listamos alguns exemplos de diversidade capturada nos vídeos térmicos:

1. **Diversidade de Cor dos Bois:** Os vídeos capturam bois com diferentes cores, incluindo brancos, pretos e manchas variadas, o que é visível mesmo nas imagens térmicas.
2. **Temperaturas Corporais Diferentes:** Variações térmicas nas imagens devido a diferenças de temperatura entre os bovinos, possivelmente devido a diferentes níveis de atividade física ou estados de saúde.
3. **Variação de Posições:** Captura de olhos em diferentes ângulos e posições, como de frente, de lado ou parcialmente cobertos.
4. **Ambiente e Iluminação:** Imagens foram capturadas sob diferentes condições de iluminação, incluindo luz direta, sombras, e reflexos no ambiente térmico.
5. **Estado dos Bovinos:** Imagens capturam diferentes estados dos bovinos, como alerta, descansando, com os olhos parcialmente fechados ou totalmente abertos.


## Processamento e Recorte de Imagens com Anotações XML


A seguir, vamos detalhar o processo de recorte e manipulação de imagens com base em anotações fornecidas em arquivos XML. Usaremos bibliotecas populares como PIL, cv2, os, e xml.etree.ElementTree para manipulação de imagens e parsing de XML.

### Importação das Bibliotecas Necessárias


- os: Usada para interagir com o sistema de arquivos, como verificar a existência de diretórios e criar novos.
- xml.etree.ElementTree: Utilizada para parsear e navegar em arquivos XML, permitindo extrair as anotações das imagens.
- PIL (Python Imaging Library): Fornece funcionalidades para carregar, manipular e salvar imagens.
- cv2: Parte do OpenCV, é usada aqui para redimensionar as imagens recortadas.
- numpy (np): Útil para conversão entre imagens PIL e arrays NumPy, especialmente durante operações de redimensionamento.

In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import cv2
import numpy as np


### Configuração dos Diretórios e Arquivos

In [60]:
frame_subfolders = [
    {'folder': 'subpasta_1', 'xml': 'annotations_1.xml', 'video_code': '00000000199000400'},
    {'folder': 'subpasta_2', 'xml': 'annotations_2.xml', 'video_code': '00000000199000400'},
    {'folder': 'subpasta_3', 'xml': 'annotations_3.xml', 'video_code': '00000000199000400'},
    {'folder': 'subpasta_4', 'xml': 'annotations_4.xml', 'video_code': '00000000199000400'},
    {'folder': 'subpasta_5', 'xml': 'annotations_5.xml', 'video_code': '00000000205000000'}
]


- **frame_subfolders**: Lista de dicionários que armazena informações sobre as subpastas de frames e seus respectivos arquivos XML de anotações. Cada item inclui:
    - **folder**: Nome da subpasta contendo os frames a serem processados.
    - **xml**: Nome do arquivo XML contendo as anotações para os frames na subpasta correspondente.
    - **video_code**: Código do vídeo associado aos frames, usado para nomear os arquivos de saída.

### Criação do Diretório de Saída

- O código verifica se o diretório de saída já existe. Se não, ele o cria e exibe uma mensagem de sucesso.

In [61]:
output_base_folder = 'labels_agrupado'
if not os.path.exists(output_base_folder):
    os.makedirs(output_base_folder)
    print(f"Pasta {output_base_folder} criada com sucesso.")

- **output_base_folder**: Define o diretório base onde as imagens processadas serão salvas.


### **Processamento de Imagens com Base em Anotações XML**

A seguir, iremos iterar sobre subpastas contendo frames de vídeo e seus respectivos arquivos XML de anotações. O objetivo é:

1. **Verificar a existência das pastas e arquivos XML**: Garantimos que tanto os frames quanto as anotações estão disponíveis antes de prosseguir.
2. **Ajustar os IDs das imagens com base nos offsets**: Cada subpasta pode começar em um frame diferente do vídeo, então ajustamos os IDs das imagens para garantir que eles correspondam corretamente ao frame original.
3. **Carregar as imagens e anotações**: As imagens são carregadas a partir dos arquivos de frames, e as anotações são extraídas do arquivo XML correspondente.
4. **Recortar e redimensionar as regiões de interesse (contextos)**: Usamos as coordenadas das caixas delimitadoras no XML para recortar partes específicas das imagens (denominadas "contextos") e redimensioná-las para um tamanho padrão de 128x128 pixels.
5. **Desenhar elementos dentro dos contextos**: Para cada "contexto" recortado, desenhamos as regiões correspondentes a "cabeças" e "olhos" para criar novas imagens anotadas, que podem ser usadas em modelos de IA.
6. **Salvar os resultados**: Tanto os contextos recortados e redimensionados quanto as imagens finais com as anotações desenhadas são salvos em diretórios específicos.

Este processo é repetido para cada subpasta e cada imagem dentro das subpastas, garantindo que todas as anotações sejam aplicadas corretamente aos frames correspondentes.

In [62]:
# Itera sobre cada subpasta de frames e seu respectivo XML
for idx, subfolder_info in enumerate(frame_subfolders):

    frames_folder = subfolder_info['folder']
    xml_file = subfolder_info['xml']
    video_code = subfolder_info['video_code']
    
    # Verifica se a subpasta e o arquivo XML existem
    if not os.path.exists(frames_folder) or not os.path.exists(xml_file):
        print(f"frames_folder: {frames_folder}")
        print(f"xml_file: {xml_file}")
        print(f"Pasta {frames_folder} ou arquivo {xml_file} não encontrado. Pulando.")
        break
    
    
    # Define o diretório de saída para a subpasta atual
    output_folder = os.path.join(output_base_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Pasta {output_folder} criada com sucesso.")

    # Parseia o arquivo XML correspondente
    tree = ET.parse(xml_file)
    root = tree.getroot()
    print(root)

    # Percorre todas as imagens descritas no XML
    for image_info in root.findall('.//image'):
        image_name = image_info.get('name')

        # Verifica se a imagem existe na pasta
        image_path = os.path.join(frames_folder, image_name)
        # print(f"Tentando acessar o arquivo: {image_path}")  # Para depuração
        if not os.path.exists(image_path):
            print(f"Imagem {image_name} não encontrada na pasta {frames_folder}. Pulando para a próxima.")
            continue

        # Carrega a imagem
        image = Image.open(image_path)

        # Contador para múltiplos contextos
        contexto_counter = 1

        # Percorre as caixas delimitadoras no XML
        for box in image_info.findall('box'):
            label = box.get('label')

            if label == "contexto":
                contexto_coords = (
                    float(box.get('xtl')),
                    float(box.get('ytl')),
                    float(box.get('xbr')),
                    float(box.get('ybr'))
                )
                cropped_image = image.crop(contexto_coords)
                
                # Redimensiona a imagem para 128x128 pixels usando OpenCV
                cropped_image_np = np.array(cropped_image)  # Converte para array NumPy
                resized_image_np = cv2.resize(cropped_image_np, (128, 128), interpolation=cv2.INTER_AREA)  # Redimensiona
                resized_image = Image.fromarray(resized_image_np)  # Converte de volta para imagem PIL

                image_name_no_ext = os.path.splitext(image_name)[0]

                cropped_image_path = os.path.join(output_folder, f"30-07-24_video_{video_code}_{image_name_no_ext}_x{contexto_counter}.png")
                resized_image.save(cropped_image_path)
                print(f"Contexto {contexto_counter} foi salvo e redimensionado para 128x128 pixels.")

                # Preparar para desenhar os elementos dentro do contexto
                contexto_width = int(contexto_coords[2] - contexto_coords[0])
                contexto_height = int(contexto_coords[3] - contexto_coords[1])
                result_image = Image.new('RGB', (contexto_width, contexto_height), (0, 0, 0))  # 'RGB' modo para cores

                draw = ImageDraw.Draw(result_image)

                # Percorrer novamente as caixas para desenhar as cabeças primeiro
                for inner_box in image_info.findall('box'):
                    inner_label = inner_box.get('label')

                    if inner_label == "cabeca":
                        cabeca_coords = (
                            float(inner_box.get('xtl')),
                            float(inner_box.get('ytl')),
                            float(inner_box.get('xbr')),
                            float(inner_box.get('ybr'))
                        )
                        relative_cabeca_coords = (
                            cabeca_coords[0] - contexto_coords[0],
                            cabeca_coords[1] - contexto_coords[1],
                            cabeca_coords[2] - contexto_coords[0],
                            cabeca_coords[3] - contexto_coords[1]
                        )
                        draw.rectangle(relative_cabeca_coords, fill=(255, 255, 255))

                # Depois, desenhar os olhos para garantir que eles fiquem por cima
                for inner_box in image_info.findall('box'):
                    inner_label = inner_box.get('label')

                    if inner_label == "olho":
                        olho_coords = (
                            float(inner_box.get('xtl')),
                            float(inner_box.get('ytl')),
                            float(inner_box.get('xbr')),
                            float(inner_box.get('ybr'))
                        )
                        relative_olho_coords = (
                            olho_coords[0] - contexto_coords[0],
                            olho_coords[1] - contexto_coords[1],
                            olho_coords[2] - contexto_coords[0],
                            olho_coords[3] - contexto_coords[1]
                        )
                        draw.rectangle(relative_olho_coords, fill=(255, 0, 0))
                #06-08-24_video2_imagem_termica2-edit_12454_y1.
                result_image = np.array(result_image)  # Converte para array NumPy
                result_image = cv2.resize(result_image, (128, 128), interpolation=cv2.INTER_AREA)  # Redimensiona
                result_image = Image.fromarray(result_image)  # Converte de volta para imagem PIL

                result_image_path = os.path.join(output_folder, f"30-07-24_video_{video_code}_{image_name_no_ext}_y{contexto_counter}.png")
                result_image.save(result_image_path)
                print(f"Imagem result.png para contexto {contexto_counter} gerada com sucesso.")

                contexto_counter += 1

<Element 'annotations' at 0x000001D06FA348B0>
Contexto 1 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 1 gerada com sucesso.
Contexto 1 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 1 gerada com sucesso.
Contexto 2 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 2 gerada com sucesso.
Contexto 1 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 1 gerada com sucesso.
Contexto 2 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 2 gerada com sucesso.
Contexto 1 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 1 gerada com sucesso.
Contexto 2 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 2 gerada com sucesso.
Contexto 1 foi salvo e redimensionado para 128x128 pixels.
Imagem result.png para contexto 1 gerada com sucesso.
Contexto 2 foi salvo e redimensionado para 128x128